In [1]:
# All code from Tutorial:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

# Part 1 : gensim LDA based on NLTK & SpaCy

# Run in python console
import nltk; nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tharsen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [4]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']
['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 't

In [5]:
%%time

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])
print('\n')

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


CPU times: user 9.01 s, sys

In [6]:
%%time

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])
print('\n')

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


CPU times: user 43.1 s, sys: 168 ms, total: 43.3 s
Wa

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
%%time
# This cell takes 2-3 minutes to run on my machine.  -j

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])
print('\n')

[['where', 's', 'thing', 'car', 'nntp_post', 'host', 'rac_wam', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'whatev', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


CPU times: user 7min 15s, sys: 1min 21s, total: 8min 36s
Wall time: 2min 23s


In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1)]]


In [10]:
mywords = []
for cp in corpus:
    for id, freq in cp:
        if id2word[id].find("_") != -1:
            mywords.append(id2word[id]) 
print(len(mywords))
print('\n')
pprint(mywords)

70469


['front_bumper',
 'maryland_college',
 'nntp_post',
 'rac_wam',
 'nntp_post',
 'carson_washington',
 'floppy_disk',
 'guy_kuo',
 'heat_sink',
 'oil_leak',
 'richardson_tx',
 'nntp_post',
 'developers_toolkit',
 'distribution_usa',
 'host_magnusug',
 'magnus_ac',
 'ohio_state',
 'delivered_gateway',
 'diamond_ss',
 'latest_driver',
 'mouse_cursor',
 'nntp_post',
 'automatic_transmission',
 'chris_silvester',
 'james_callison',
 'texas_instrument',
 'nntp_post',
 'ohio_state',
 'acs_ohio',
 'asking_shipping',
 'bottom_magnus',
 'nntp_post',
 'evanston_illinois',
 'northwestern_university',
 'bob_beauchaine',
 'bronx_away',
 'death_penalty',
 'innocents_die',
 'queens_could',
 'robert_beauchaine',
 'sank_manhattan',
 'success_rate',
 'nntp_post',
 'distribution_world',
 'frank_odwyer',
 'host_solntze',
 'jon_livesey',
 'wpd_sgi',
 'nntp_post',
 'dsto_gov',
 'tv_station',
 'west_coast',
 'years_ago',
 'bob_gaj',
 'bob_gajarsky',
 'curtis_jackson',
 'flame_war',
 'lotsa_point',
 'mo

 'eleven_nigel',
 'handson_experience',
 'megatek_corporation',
 'pls_kindly',
 'randy_davis',
 'ucsd_megatek',
 'zx_pilot',
 'nntp_post',
 'po_andrew',
 'engineering_carnegie',
 'mellon_pittsburgh',
 'original_packag',
 'doug_dolven',
 'mel_hall',
 'pavel_bure',
 'panix_public',
 'jesus_christ',
 'oregon_health',
 'holy_spirit',
 'bible_teach',
 'portland_oregon',
 'spiritual_gift',
 'virgin_mary',
 'nntp_post',
 'floppy_disk',
 'distribution_world',
 'new_york',
 'jet_propulsion',
 '_',
 'plus_shipp',
 'comp_graphic',
 'public_domain',
 'ames_arc',
 'nasa_gov',
 'map_projection',
 'frequently_ask',
 'archive_name',
 'center_moffett',
 'chapel_hill',
 'customer_service',
 'geological_survey',
 'host_mahler',
 'ibm_pc',
 'jon_leech',
 'last_modifi',
 'nasa_ames',
 'naval_observatory',
 'north_carolina',
 'planetary_position',
 'three_dimensional',
 'turbo_pascal',
 'willmann_bell',
 'nntp_post',
 'latest_driver',
 'ftp_site',
 'nntp_post',
 'bush_administration',
 'clinton_administrati

 'host_localhost',
 'mcsnet_contributor',
 'nntp_post',
 'nntp_post',
 'handson_experience',
 'pls_kindly',
 'usr_lib',
 'nntp_post',
 'last_night',
 'rec_sport',
 'red_wing',
 'adams_division',
 'adirondack_gm',
 'adirondack_r',
 'ahl_mail',
 'baltimore_skipjack',
 'binghamton_gm',
 'binghamton_ranger',
 'binghamton_utica',
 'boston_bruin',
 'bri_farenell',
 'calder_cup',
 'cape_breton',
 'cdi_gm',
 'champions_phoenix',
 'champs_ahl',
 'champs_clarkson',
 'division_champion',
 'ecac_contact',
 'final_standing',
 'fredericton_canadien',
 'glens_fall',
 'gm_providence',
 'hockey_adirondack',
 'hockey_ecac',
 'list_congrat',
 'logic_clarkson',
 'maple_leafs',
 'moncton_gm',
 'moncton_hawk',
 'providence_bruin',
 'providence_gm',
 'springfield_indian',
 'springfield_providence',
 'suns_pacific',
 'tournament_champ',
 'utica_gm',
 'around_jupiter',
 'jupiter_radii',
 'mark_brader',
 'temporary_orbit',
 'arizona_tucson',
 'compton_gamma',
 'laboratory_tucson',
 'lunar_planetary',
 'neil_geh

 'finance_guy',
 'nissan_sentra',
 'service_scam',
 'david_veal',
 'health_scienc',
 'individual_liberty',
 'paul_prescod',
 'cochrane_jame',
 'attitude_toward',
 'south_florida',
 'nntp_post',
 'distribution_world',
 'aaron_ray',
 'special_investor',
 'stratus_computer',
 'wtc_bomb',
 'nntp_post',
 'distribution_world',
 'tim_ciceran',
 'st_catharine',
 'massachusetts_institute',
 'mark_ashley',
 'aaron_bryce',
 'daily_basis',
 'paul_conditt',
 'socially_unacceptable',
 'dale_cook',
 'hypocrisy_t',
 'neighbor_political',
 'ryan_scharfy',
 'smoke_pot',
 'trouble_aim',
 'thomas_wri',
 'mohit_goyal',
 'line_communication',
 'services_gu',
 'buslogic_card',
 'toshiba_cdrom',
 'nntp_post',
 'case_western',
 'cleveland_oh',
 'reserve_university',
 'cleveland_ohio',
 'thor_in',
 'robert_novitskey',
 'grad_student',
 'josh_hopkin',
 'recently_bought',
 'distribution_world',
 'greatly_appreciat',
 'opposite_direction',
 'simple_minded',
 'georgia_athen',
 'paul_hudson',
 'nntp_posting',
 'rada

 'moink_nmsu',
 'escrow_hous',
 'executive_branch',
 'richard_nixon',
 'nntp_post',
 'apr_gmt',
 'krakatoa_mailer',
 'kupajava_east',
 'psilink_do',
 'masud_khan',
 'robert_knowl',
 'nntp_post',
 'newsreader_tin',
 'western_australia',
 'serial_port',
 'loopback_connector',
 'iowa_state',
 'second_amendment',
 'nuclear_weapon',
 'dan_sorenson',
 'depends_upon',
 'dod_isu',
 'exciting_unusual',
 'exotic_distant',
 'foxvog_dougla',
 'machines_meet',
 'mass_destruction',
 'portal_system',
 'press_conference',
 'barbecued_food',
 'health_risk',
 'robert_thorson',
 'duncan_hine',
 'earl_grey',
 'nntp_post',
 'thou_shalt',
 'news_gateway',
 'jim_meritt',
 'years_ago',
 'last_night',
 'boeing_comput',
 'keywords_brick',
 'neil_william',
 'toyota_pickup',
 'brian_larose',
 'apostle_paul',
 'georgia_athen',
 'ted_kalivoda',
 'temporary_orbit',
 'ancient_mayan',
 'phil_fraer',
 'phil_fraering',
 'southwestern_louisiana',
 'televison_repo',
 'moon_land',
 'lunar_orbit',
 'distribution_na',
 'red_

...

 'go_ahead',
 'line_communication',
 'services_gu',
 'black_market',
 'crypto_anarchy',
 'digital_pseudonyms',
 'knowledge_reputation',
 'mailsafe_available',
 'markets_apto',
 'melbourne_australia',
 'monash_university',
 'ms_window',
 'michael_panayiotaki',
 'melb_australia',
 'grp_file',
 'exit_code',
 'my_int_var',
 'works_fine',
 'penn_state',
 'frank_crary',
 'internal_passport',
 'six_month',
 'emergency_medical',
 'paul_havemann',
 'newsreader_tin',
 'opel_owner',
 'matthew_macintyre',
 'serial_number',
 'sales_tax',
 'voice_fax',
 'caveat_emptor',
 'nntp_post',
 'science_fiction',
 'santa_cruz',
 'marcus_lindroo',
 'plus_shipping',
 'nntp_post',
 'lance_colostate',
 'tim_clock',
 'basil_hamdan',
 'tv_station',
 'flame_thrower',
 'carlos_carrion',
 'inflaming_passion',
 'costa_mesa',
 'candida_albican',
 'steve_dyer',
 'fair_amount',
 'consulting_cambridge',
 'harvard_rayssd',
 'linus_spdcc',
 'daily_basis',
 'anal_retentive',
 'nntp_posting',
 'nick_pettefar',
 'tin_version',


 'richard_warner',
 'ftp_cica',
 'math_cs',
 'jorge_lach',
 'al_devilbiss',
 'computing_center',
 'hi_r',
 'hi_re',
 'jeff_hite',
 'service_provider',
 'nntp_post',
 'wants_convertible',
 'keith_nuetzman',
 'host_cunixb',
 'nntp_posting',
 'peter_garfiel',
 'distribution_usa',
 'please_respond',
 'vms_vnews',
 'flight_center',
 'shuttle_launch',
 'nntp_post',
 'cs_dept',
 'distribution_inet',
 'homeopathy_tradition',
 'avoiding_mistak',
 'lee_lady',
 'sci_med',
 'sci_psychology',
 'russell_turpin',
 'errors_turpin',
 'nntp_post',
 'vesa_local',
 'video_card',
 'local_bu',
 'oklahoma_norman',
 'diamond_stealth',
 'mb_ram',
 'ecn_uoknor',
 'ati_graphic',
 'stealth_vlb',
 'years_ago',
 'policy_research',
 'united_state',
 'cdp_nf',
 'cdp_uucp',
 'cpr_apr',
 'human_right',
 'nf_id',
 'human_being',
 'elias_davidsson',
 'middle_east',
 'unconventional_peace',
 'peaceful_solution',
 'mixed_marriage',
 'mixed_marriag',
 'mixed_stock',
 'box_reykjavik',
 'proposal_unconventional',
 'affirmativ

 'art_microcircuit',
 'cabinet_official',
 'consider_incorporat',
 'frequent_consultation',
 'global_marketplace',
 'images_hdtv',
 'information_superhighway',
 'mat_heyman',
 'permitting_wider',
 'presidents_directive',
 'wireless_communication',
 'totally_unorganized',
 'nntp_post',
 'distribution_world',
 'works_fine',
 'vms_vnew',
 'rosie_uh',
 'data_strobe',
 'motorola_xc',
 'nntp_post',
 'newsreader_tin',
 '_',
 'athena_widget',
 'xlib_xt',
 'comp_sy',
 'ibm_rs',
 'mix_gl',
 'suresh_thennarangam',
 'larry_pyeatt',
 'nntp_post',
 'keith_allan',
 'political_atheist',
 'case_western',
 'reserve_university',
 'bobby_mozumder',
 'keith_ryan',
 'student_cwru',
 'virile_man',
 'amateur_radio',
 'apr_gmt',
 'radio_shack',
 'douglas_rand',
 'osf_motif',
 'ham_radio',
 'randall_rhea',
 'radio_operator',
 'signal_surge',
 'nntp_post',
 'distribution_world',
 'nntp_posting',
 'robert_hite',
 'philadelphia_phillie',
 'pitching_staff',
 'spring_train',
 'nntp_post',
 'latest_driver',
 'new_yor

 'dave_duff',
 'nntp_post',
 'host_enterpoop',
 'windows_nt',
 'adam_adam',
 'distribution_usa',
 'digital_equipment',
 'opinions_expressed',
 'sb_pro',
 'henry_spencer',
 'allen_sherzer',
 'coffee_churchill',
 'evil_genius',
 'cost_estimate',
 'warrent_develop',
 'working_ssto',
 'moon_resident',
 'allen_lady',
 'brian_kendig',
 'holy_spirit',
 'gods_promise',
 'san_francisco',
 'brian_ceccarelli',
 'comme_aucun',
 'croire_netre',
 'ends_rousseau',
 'pas_mieux',
 'que_jai',
 'suis_fait',
 'vus_jose',
 'starfleet_headquarter',
 'sampling_rate',
 'nntp_post',
 'okcforum_osrhe',
 'okcforum_unix',
 'islamic_authority',
 'bill_conner',
 'computer_science',
 '_',
 'wiretap_chip',
 'graham_toal',
 'corporate_acceptance',
 'nntp_post',
 'distribution_world',
 'los_angel',
 'dev_null',
 'user_interface',
 'event_handler',
 'virtual_reality',
 'held_responsible',
 'floppy_disk',
 'distribution_world',
 'clipper_chip',
 'serial_number',
 'session_key',
 'key_escrow',
 'dorothy_denn',
 'escrow_ag

 'hp_hut',
 'makes_sense',
 'gerald_olchowy',
 'toronto_chemistry',
 'press_conference',
 'nelson_lu',
 'mike_keenan',
 'mark_messier',
 'rangers_messier',
 'nntp_post',
 'internet_uucp',
 'isc_rit',
 'vancouver_canada',
 'link_british',
 'tammy_healy',
 'walla_walla',
 'apr_gmt',
 'benedikt_rosenau',
 'roman_catholic',
 'generally_accept',
 'middle_ag',
 'msg_sensitivity',
 'flight_center',
 'nasa_goddard',
 'nntp_post',
 'nsw_australia',
 'ray_traced',
 'nntp_post',
 'federal_government',
 'new_york',
 'united_state',
 'los_angele',
 'talk_politic',
 'po_box',
 'reagan_bush',
 'thirty_year',
 'adolf_hitler',
 'mein_kampf',
 'rush_limbaugh',
 'socialized_medicine',
 'gulf_war',
 'aryan_race',
 'judeo_christian',
 'divinely_inspir',
 'matt_freivald',
 'case_western',
 'practical_purpos',
 'cleveland_ohio',
 'human_being',
 'strongly_suspect',
 'applied_physic',
 'graduate_student',
 'absolute_truth',
 'electrical_engg',
 'mans_intimidat',
 'velasco_jr',
 'virgilio_dean',
 'wannabee_bul

In [11]:
id2word[0]

'addition'

In [12]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_post', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('whatev', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [13]:
%%time
# This cell takes about 4 minutes to run on my machine.  -j

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

CPU times: user 8min 20s, sys: 1min 22s, total: 9min 42s
Wall time: 4min 8s


In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.094*"space" + 0.044*"image" + 0.025*"display" + 0.021*"blue" + '
  '0.019*"earth" + 0.018*"print" + 0.017*"research" + 0.016*"project" + '
  '0.016*"satellite" + 0.015*"tank"'),
 (1,
  '0.050*"food" + 0.045*"bus" + 0.044*"specifically" + 0.036*"msg" + '
  '0.030*"eat" + 0.030*"motorcycle" + 0.025*"ride" + 0.018*"confuse" + '
  '0.016*"entry" + 0.016*"shift"'),
 (2,
  '0.029*"use" + 0.027*"system" + 0.021*"window" + 0.018*"card" + 0.016*"file" '
  '+ 0.016*"run" + 0.014*"program" + 0.013*"problem" + 0.012*"also" + '
  '0.011*"need"'),
 (3,
  '0.080*"game" + 0.076*"team" + 0.045*"play" + 0.045*"win" + 0.044*"player" + '
  '0.036*"hockey" + 0.030*"season" + 0.026*"contact" + 0.020*"goal" + '
  '0.020*"fan"'),
 (4,
  '0.112*"circuit" + 0.018*"stable" + 0.012*"cam" + 0.011*"chemistry" + '
  '0.010*"scout" + 0.007*"tran" + 0.006*"arabic" + 0.005*"randomly" + '
  '0.004*"axis" + 0.000*"tablet"'),
 (5,
  '0.129*"gun" + 0.052*"_" + 0.036*"weapon" + 0.025*"discipline" + '
  '0.023*"cri

In [15]:
%%time

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
print('\n')


Perplexity:  -14.792846126242049

Coherence Score:  0.5025181203779839


CPU times: user 27.2 s, sys: 5.3 s, total: 32.5 s
Wall time: 28 s


In [16]:
%%time

# Visualize the topics
#
# If you get an error like this: "pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. 
# A future version of pandas will change to not sort by default."
#
# then from the command line do: "pip install pandas==0.21.0"

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)  # sort=False ? sort=True
vis
print('\n')



CPU times: user 9.56 s, sys: 2.29 s, total: 11.8 s
Wall time: 10.9 s


In [17]:
#import pandas as pd
#pd.__version__

vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      26.783401        1       1 -0.361834 -0.070399
13     22.668081        1       2 -0.353337 -0.124224
12     12.952993        1       3 -0.296754 -0.075625
2      10.931390        1       4 -0.267229 -0.027507
15      8.618702        1       5 -0.205850  0.115204
6       5.042089        1       6 -0.094984  0.441082
17      3.256453        1       7 -0.138349 -0.079390
3       1.988039        1       8 -0.025693  0.031644
0       1.898906        1       9  0.026200  0.030532
16      1.504176        1      10  0.041434  0.018806
5       1.296365        1      11  0.130633 -0.026200
8       0.645863        1      12  0.137391 -0.009256
18      0.562713        1      13  0.145351 -0.015355
1       0.504953        1      14  0.153874 -0.020524
11      0.369607        1      15  0.170989 -0.025604
9       0.297865        1      16  0.178093 -0.028675
19      0.295467        1      17  0.182553 -0.031819
10      0.248325        1      18  0.176187 -0.028066
4       0.074328        1      19  0.199575 -0.036905
14      0.060280        1      20  0.201750 -0.037718, topic_info=      Category          Freq                 Term         Total  loglift  \
term                                                                       
4146   Default  64902.000000                   ax  64902.000000  30.0000   
82     Default  24205.000000                  not  24205.000000  29.0000   
21     Default  18730.000000                 line  18730.000000  28.0000   
30     Default  16710.000000         organization  16710.000000  27.0000   
65     Default  15422.000000                   do  15422.000000  26.0000   
46     Default   7809.000000           university   7809.000000  25.0000   
322    Default  13744.000000                write  13744.000000  24.0000   
244    Default  11807.000000                   be  11807.000000  23.0000   
154    Default  17232.000000                would  17232.000000  22.0000   
3721   Default   4706.000000                  max   4706.000000  21.0000   
29     Default   6772.000000            nntp_post   6772.000000  20.0000   
16     Default   6320.000000                 host   6320.000000  19.0000   
55     Default  10769.000000              article  10769.000000  18.0000   
2191   Default   2698.000000                  gun   2698.000000  17.0000   
50     Default   5516.000000                 year   5516.000000  16.0000   
1911   Default   2883.000000                space   2883.000000  15.0000   
303    Default  10381.000000                  say  10381.000000  14.0000   
206    Default   5392.000000               system   5392.000000  13.0000   
18     Default  10702.000000                 know  10702.000000  12.0000   
317    Default   7222.000000                  use   7222.000000  11.0000   
271    Default   7960.000000                 good   7960.000000  10.0000   
2726   Default   2570.000000                 game   2570.000000   9.0000   
567    Default   2443.000000                 team   2443.000000   8.0000   
263    Default   1482.000000                drive   1482.000000   7.0000   
226    Default   8830.000000                   go   8830.000000   6.0000   
756    Default   2031.000000                  key   2031.000000   5.0000   
43     Default   3697.000000                thank   3697.000000   4.0000   
967    Default   2371.000000            christian   2371.000000   3.0000   
215    Default   3680.000000               window   3680.000000   2.0000   
147    Default   7793.000000                think   7793.000000   1.0000   
...        ...           ...                  ...           ...      ...   
1825   Topic20      9.239208        demonstration     10.356578   7.2998   
2434   Topic20      5.612238  bush_administration      6.729608   7.2324   
3616   Topic20      4.667922        qualification      5.785292   7.1993   
15273  Topic20      0.011483     